In [1]:
#!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
#!pip install --upgrade google-cloud-bigquery

In [2]:
import os
import bq_helper
from bq_helper import BigQueryHelper
import pandas as pd
%run country-boundingbox.ipynb

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Data-mining-afa11af25388.json"
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
noaa = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="ghcn_d")

In [55]:
def weatherDataByCountry(countryID, startYear, endYear):
    coordinates = country_bounding_boxes[countryID][1]
    lonMin = str(coordinates[0])
    latMin = str(coordinates[1])
    lonMax = str(coordinates[2])
    latMax = str(coordinates[3])
    weatherData = pd.DataFrame()
    for i in range(startYear,endYear):
        query = """SELECT
          stations.id,
          weather.date,
          weather.element,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather INNER JOIN `bigquery-public-data.ghcn_d.ghcnd_stations` AS stations ON 
          weather.id = stations.id
        WHERE
          weather.element = 'TAVG'
          AND stations.latitude >"""+latMin+"""
          AND stations.latitude <"""+latMax+"""
          AND stations.longitude >"""+lonMin+"""
          AND stations.longitude <"""+lonMax+"""
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        response2.dropna()
        weatherData = weatherData.append(response2)
    return weatherData

In [56]:
dfWeather= weatherDataByCountry('HN', 2013, 2015)
print(dfWeather.size)
print(dfWeather.head(10))

13964
            id        date element  value
0  HOM00078705  2013-01-01    TAVG  267.0
1  HOM00078708  2013-01-01    TAVG  249.0
2  ES003800510  2013-01-01    TAVG  263.0
3  HO000078720  2013-01-01    TAVG  194.0
4  ES003800510  2013-01-02    TAVG  262.0
5  HOM00078708  2013-01-02    TAVG  261.0
6  HOM00078705  2013-01-02    TAVG  253.0
7  HO000078711  2013-01-02    TAVG  259.0
8  HO000078720  2013-01-02    TAVG  209.0
9  HOM00078708  2013-01-03    TAVG  252.0


In [60]:
def closestStationToLoc(lat, long, startYear, endYear ):
    query = """SELECT weather.id, POWER(POWER(weather.latitude - """+str(lat)+""",2) + POWER(weather.longitude - """+str(long)+""",2), 1/2) AS distance, weather.latitude, weather.longitude
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS weather
    ORDER BY
        distance ASC
    LIMIT 10;
            """
    response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
    return weatherDataForStationID(response2['id'].iloc[0], startYear, endYear )

def weatherDataForStationID(ID, startYear, endYear):
    weatherData = pd.DataFrame()
    for i in range(startYear,endYear):
        query = """SELECT
          weather.date,
          weather.value
        FROM
          `bigquery-public-data.ghcn_d.ghcnd_"""+str(i)+"""` AS weather
        WHERE
            weather.element = 'TAVG'
            AND weather.id = '"""+ID+"""'
        ORDER BY
            weather.date ASC;
                """
        response2 = noaa.query_to_pandas_safe(query, max_gb_scanned=10)
        response2.dropna()
        weatherData = weatherData.append(response2)
    return weatherData

In [61]:
dfWeather = closestStationToLoc(15.2000, -86.2419, 2013, 2015)
print(dfWeather.size)
print(dfWeather.head(10))

1448
         date  value
0  2013-01-01  267.0
1  2013-01-02  253.0
2  2013-01-03  251.0
3  2013-01-04  238.0
4  2013-01-05  251.0
5  2013-01-06  251.0
6  2013-01-07  248.0
7  2013-01-08  254.0
8  2013-01-09  258.0
9  2013-01-10  258.0
